In [1]:
import sys
import xarray as xr
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import glob
from netCDF4 import Dataset
import cartopy.crs as ccrs
import os

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%autosave 60

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


Autosaving every 60 seconds


In [2]:
sys.path.insert(0,"/scratch/cnt0024/hmg2840/albert7a/DEV/git/powerspec/powerspec")
import powerspec as pp

In [3]:
sys.path.insert(0,"/scratch/cnt0024/hmg2840/albert7a/DEV/git/diags-CMEMS-on-occigen/common-lib/")
import GriddedData
import WavenumberSpectrum as ws


In [4]:
# Some functions that allow us to compute spectra in boxes

##########################################
def get_values_in_box(imin,imax,jmin,jmax,data):
    values = data[:,jmin:jmax+1,imin:imax+1]
    values = ma.masked_invalid(values)
    return values


##########################################
def compute_spec_for_box(imin,imax,jmin,jmax,data,Mth,navlon,navlat,name,WaveSpecResult,param):
    var = get_values_in_box(imin,imax,jmin,jmax,data)
    pspec,kspec = compute_spec(var,navlon,navlat)
    np.savez(WaveSpecResult+'WaveSpec_'+name+'_'+param+'_'+Mth, kspec=kspec ,pspec=pspec)

##########################################
def compute_spec(data,navlon,navlat):
    days = len(data)
    mth_pspec = []
    for it in np.arange(0,days):
        arr = data[it]
        datab = arr.squeeze()
        pspec,kstep = pp.wavenumber_spectra(datab,navlon,navlat)
        mth_pspec.append(pspec)
    mthly_pspec = np.array(mth_pspec)
    mean_mthly_pspec = mthly_pspec.mean(axis=0)
    return kstep,mean_mthly_pspec

In [48]:
def compute_kespec(box):
    YrMth  = ['2012m10','2012m11','2012m12','2013m01','2013m02','2013m03','2013m04','2013m05','2013m06','2013m07','2013m08','2013m09']
    # Folders containing U and V spectral
    u_database = '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Uspec/'
    v_database = '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Vspec/'    # Folders to contain computed KE spectral
    KEspecFolder = '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-KEspec/'    
    !mkdir -p $KEspecFolder
    u_filenames = sorted(glob.glob(u_database + 'WaveSpec_'+str(box)+'_Uspec_*.npz'))
    v_filenames = sorted(glob.glob(v_database + 'WaveSpec_'+str(box)+'_Vspec_*.npz'))
    for i in range(len(u_filenames)):
        if not os.path.exists(KEspecFolder+'WaveSpec_'+str(box)+'_KEspec_'+YrMth[i]+'.npz'):
            print(KEspecFolder+'WaveSpec_'+str(box)+'_KEspec_'+YrMth[i]+'.npz')
            uspec = np.load(u_filenames[i])['pspec']
            vspec = np.load(v_filenames[i])['pspec']
            kspec = np.load(u_filenames[i])['kspec']
            KEspec = 0.5*(uspec + vspec)
            print(KEspecFolder+'WaveSpec_'+str(box)+'_KEspec_'+YrMth[i]+'.npz')
            np.savez(KEspecFolder+'WaveSpec_'+str(box)+'_KEspec_'+YrMth[i]+'.npz',kspec = kspec,KEspec = KEspec)

In [49]:
compute_kespec('box1')

/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-KEspec/WaveSpec_box1_KEspec_2013m04.npz


ValueError: operands could not be broadcast together with shapes (323,) (321,) 

In [52]:
box='box1'

In [50]:
i=3

In [53]:
    YrMth  = ['2012m10','2012m11','2012m12','2013m01','2013m02','2013m03','2013m04','2013m05','2013m06','2013m07','2013m08','2013m09']
    # Folders containing U and V spectral
    u_database = '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Uspec/'
    v_database = '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Vspec/'    # Folders to contain computed KE spectral
    KEspecFolder = '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-KEspec/'    
    !mkdir -p $KEspecFolder
    u_filenames = sorted(glob.glob(u_database + 'WaveSpec_'+str(box)+'_Uspec_*.npz'))
    v_filenames = sorted(glob.glob(v_database + 'WaveSpec_'+str(box)+'_Vspec_*.npz'))


In [54]:
            uspec = np.load(u_filenames[i])['pspec']
            vspec = np.load(v_filenames[i])['pspec']


In [55]:
uspec.shape

(323,)

In [56]:
vspec.shape

(321,)

In [57]:
        imin=688
        imax=1296
        jmin=235
        jmax=1004


In [89]:
    param='Uspec'
    varname = 'vozocrtx'
    WaveSpecResult = '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-'+str(param)+'/'
    !mkdir -p $WaveSpecResult
    YrMth  = ['2012m10','2012m11','2012m12','2013m01','2013m02','2013m03','2013m04','2013m05','2013m06','2013m07','2013m08','2013m09']



In [90]:
ii=3

In [109]:
            filename='/scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/GS/NATL60GS-CJM165_y'+str(YrMth[ii])+'d??.1d_gridUsurf.nc'
            print(filename)
            ds = xr.open_mfdataset(filename,chunks={'x':500,'y':500,'time_counter':1})
            data = ds[varname]
            if len(ds.nav_lon.dims) == 3:
                navlon=ds.nav_lon[0,imin:imax+1,jmin:jmax+1]
                navlat=ds.nav_lat[0,imin:imax+1,jmin:jmax+1]
            if len(ds.nav_lon.dims) == 2:
                navlon=ds.nav_lon[imin:imax+1,jmin:jmax+1]
                navlat=ds.nav_lat[imin:imax+1,jmin:jmax+1]


/scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/GS/NATL60GS-CJM165_y2013m01d??.1d_gridUsurf.nc


In [100]:
    var = get_values_in_box(imin,imax,jmin,jmax,data)


In [101]:
    days = len(var)
    mth_pspec = []


In [102]:
print(days)


31


In [103]:
it=0

In [104]:
        arr = var[it]
        datab = arr.squeeze()


In [105]:
datab.shape

(770, 609)

In [106]:
        pspec,kstep = pp.wavenumber_spectra(datab,navlon,navlat)


In [107]:
pspec.shape

(323,)

In [69]:
pspec

array([1.10129854e-06, 2.20259708e-06, 3.30389562e-06, 4.40519416e-06,
       5.50649271e-06, 6.60779125e-06, 7.70908979e-06, 8.81038833e-06,
       9.91168687e-06, 1.10129854e-05, 1.21142840e-05, 1.32155825e-05,
       1.43168810e-05, 1.54181796e-05, 1.65194781e-05, 1.76207767e-05,
       1.87220752e-05, 1.98233737e-05, 2.09246723e-05, 2.20259708e-05,
       2.31272694e-05, 2.42285679e-05, 2.53298664e-05, 2.64311650e-05,
       2.75324635e-05, 2.86337621e-05, 2.97350606e-05, 3.08363592e-05,
       3.19376577e-05, 3.30389562e-05, 3.41402548e-05, 3.52415533e-05,
       3.63428519e-05, 3.74441504e-05, 3.85454489e-05, 3.96467475e-05,
       4.07480460e-05, 4.18493446e-05, 4.29506431e-05, 4.40519416e-05,
       4.51532402e-05, 4.62545387e-05, 4.73558373e-05, 4.84571358e-05,
       4.95584343e-05, 5.06597329e-05, 5.17610314e-05, 5.28623300e-05,
       5.39636285e-05, 5.50649271e-05, 5.61662256e-05, 5.72675241e-05,
       5.83688227e-05, 5.94701212e-05, 6.05714198e-05, 6.16727183e-05,
      

In [70]:
kstep.shape

(323,)

In [71]:
kstep

array([2.46922098e+01, 4.38128257e+02, 9.21572367e+02, 9.44442324e+02,
       6.87919506e+02, 4.58993847e+02, 3.78902763e+02, 3.28743683e+02,
       2.45674544e+02, 2.19620957e+02, 1.56227426e+02, 1.07299464e+02,
       1.15630862e+02, 1.11058289e+02, 9.09702040e+01, 5.97053724e+01,
       5.89044591e+01, 4.42636310e+01, 3.89739579e+01, 2.66598784e+01,
       2.31868879e+01, 3.01067684e+01, 1.74979579e+01, 1.48196861e+01,
       1.58574944e+01, 1.11339401e+01, 1.29853441e+01, 1.15601788e+01,
       8.61458213e+00, 8.64177789e+00, 8.16184942e+00, 5.62157930e+00,
       7.39321413e+00, 6.78441640e+00, 5.07583235e+00, 4.73072299e+00,
       5.77807129e+00, 4.92373440e+00, 4.51412863e+00, 4.65624610e+00,
       4.20031218e+00, 3.66004702e+00, 2.66025496e+00, 3.39265920e+00,
       2.35552455e+00, 3.27040709e+00, 2.67233006e+00, 2.26322460e+00,
       2.16913436e+00, 2.03238403e+00, 1.46868007e+00, 1.80462929e+00,
       1.57699512e+00, 1.34547986e+00, 1.42847151e+00, 1.24230739e+00,
      

In [110]:
ii=3
filename='/scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/GS/NATL60GS-CJM165_y'+str(YrMth[ii])+'d??.1d_gridVsurf.nc'
print(filename)
ds = xr.open_mfdataset(filename,chunks={'x':500,'y':500,'time_counter':1})
data = ds['vomecrty']
if len(ds.nav_lon.dims) == 3:
    navlon=ds.nav_lon[0,imin:imax+1,jmin:jmax+1]
    navlat=ds.nav_lat[0,imin:imax+1,jmin:jmax+1]
if len(ds.nav_lon.dims) == 2:
    navlon=ds.nav_lon[imin:imax+1,jmin:jmax+1]
    navlat=ds.nav_lat[imin:imax+1,jmin:jmax+1]
var = get_values_in_box(imin,imax,jmin,jmax,data)
it=0
arr = var[it]
datab = arr.squeeze()
pspec,kstep = pp.wavenumber_spectra(datab,navlon,navlat)
print(pspec.shape)

/scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/GS/NATL60GS-CJM165_y2013m01d??.1d_gridVsurf.nc
(323,)


In [74]:
for it in np.arange(0,days):
    arr = var[it]
    datab = arr.squeeze()
    pspec,kstep = pp.wavenumber_spectra(datab,navlon,navlat)
    print(it,' ',pspec.shape)

0   (323,)
1   (323,)
2   (323,)
3   (323,)
4   (323,)
5   (323,)
6   (323,)
7   (323,)
8   (323,)
9   (323,)
10   (323,)
11   (323,)
12   (323,)
13   (323,)
14   (323,)
15   (323,)
16   (323,)
17   (323,)
18   (323,)
19   (323,)
20   (323,)
21   (323,)
22   (323,)
23   (323,)
24   (323,)
25   (323,)
26   (323,)
27   (323,)
28   (323,)
29   (323,)


In [111]:
def check_size(box):
    YrMth  = ['2013m01','2013m02','2013m03','2013m04','2013m05','2013m06','2013m07','2013m08','2013m09','2012m10','2012m11','2012m12']
    # Folders containing U and V spectral
    u_database = '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Uspec/'
    v_database = '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Vspec/'    # Folders to contain computed KE spectral
    KEspecFolder = '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-KEspec/'    
    !mkdir -p $KEspecFolder
    u_filenames = sorted(glob.glob(u_database + 'WaveSpec_'+str(box)+'_Uspec_*.npz'))
    v_filenames = sorted(glob.glob(v_database + 'WaveSpec_'+str(box)+'_Vspec_*.npz'))
    for i in range(len(u_filenames)):
        print('i = ',i)
        print(u_filenames[i])
        uspec = np.load(u_filenames[i])['pspec']
        print(v_filenames[i])
        vspec = np.load(v_filenames[i])['pspec']
        print('uspec : ',uspec.shape)
        print('vspec : ',vspec.shape)

In [113]:
check_size('box1')

i =  0
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Uspec/WaveSpec_box1_Uspec_2012m10.npz
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Vspec/WaveSpec_box1_Vspec_2012m10.npz
uspec :  (321,)
vspec :  (321,)
i =  1
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Uspec/WaveSpec_box1_Uspec_2012m11.npz
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Vspec/WaveSpec_box1_Vspec_2012m11.npz
uspec :  (321,)
vspec :  (321,)
i =  2
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Uspec/WaveSpec_box1_Uspec_2012m12.npz
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Vspec/WaveSpec_box1_Vspec_2012m12.npz
uspec :  (321,)
vspec :  (321,)
i =  3
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Uspec/WaveSpec_box1_Uspec_2013m01.npz
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Vspec/WaveSpec_box1_Vspec_2013m01.npz
uspec :  (323,)
vspec :  (321,)
i =  4
/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Uspec/WaveSpec_box1_Uspec_2013m02.npz


In [81]:
    param='Vspec'
    varname = 'vomecrty'
    WaveSpecResult = '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-'+str(param)+'/'
    !mkdir -p $WaveSpecResult
    YrMth  = ['2013m01','2013m02','2013m03','2013m04','2013m05','2013m06','2013m07','2013m08','2013m09','2012m10','2012m11','2012m12']

    ii=3
    Mth = YrMth[ii]

    filename='/scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/GS/NATL60GS-CJM165_y'+str(YrMth[ii])+'d??.1d_gridVsurf.nc'
    ds = xr.open_mfdataset(filename,chunks={'x':500,'y':500,'time_counter':1})
    data = ds[varname]
    if len(ds.nav_lon.dims) == 3:
        navlon=ds.nav_lon[0,imin:imax+1,jmin:jmax+1]
        navlat=ds.nav_lat[0,imin:imax+1,jmin:jmax+1]
    if len(ds.nav_lon.dims) == 2:
        navlon=ds.nav_lon[imin:imax+1,jmin:jmax+1]
        navlat=ds.nav_lat[imin:imax+1,jmin:jmax+1]
    var = get_values_in_box(imin,imax,jmin,jmax,data)
    pspec,kspec = compute_spec(var,navlon,navlat)
    print(pspec.shape)

(323,)


In [83]:
    name=box
    np.savez(WaveSpecResult+'WaveSpec_'+name+'_'+param+'_'+Mth, kspec=kspec ,pspec=pspec)


In [85]:
print(WaveSpecResult+'WaveSpec_'+name+'_'+param+'_'+Mth)

/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Vspec/WaveSpec_box1_Vspec_2013m04


In [86]:
    v_filenames = sorted(glob.glob(v_database + 'WaveSpec_'+str(box)+'_Vspec_*.npz'))


In [87]:
        i=3
        print(v_filenames[i])
        uspec = np.load(u_filenames[i])['pspec']
        vspec = np.load(v_filenames[i])['pspec']
        print('uspec : ',uspec.shape)
        print('vspec : ',vspec.shape)

/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Vspec/WaveSpec_box1_Vspec_2013m01.npz
uspec :  (323,)
vspec :  (321,)


In [88]:
print(u_filenames)

['/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Uspec/WaveSpec_box1_Uspec_2012m10.npz', '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Uspec/WaveSpec_box1_Uspec_2012m11.npz', '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Uspec/WaveSpec_box1_Uspec_2012m12.npz', '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Uspec/WaveSpec_box1_Uspec_2013m01.npz', '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Uspec/WaveSpec_box1_Uspec_2013m02.npz', '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Uspec/WaveSpec_box1_Uspec_2013m03.npz', '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Uspec/WaveSpec_box1_Uspec_2013m04.npz', '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Uspec/WaveSpec_box1_Uspec_2013m05.npz', '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Uspec/WaveSpec_box1_Uspec_2013m06.npz', '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Uspec/WaveSpec_box1_Uspec_2013m07.npz', '/scratch/cnt0024/hmg2840/albert7a/NATL